In [72]:
!pip install minsearch  groq load_dotenv


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [1]:
import minsearch
import json 

In [2]:
with open('documents.json' , 'rt') as f_in:
    docs_raw = json.load(f_in)
    

In [3]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)


In [4]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [5]:
index = minsearch.Index(
    text_fields=["question" , "text" ,"section"],
    keyword_fields= ["course"]
)

In [10]:
    SELECT * WHERE Course = 'data-engineering-zoomcamp';


SyntaxError: invalid syntax (3333116107.py, line 1)

In [6]:
q = 'the course has already started, can I still enroll?'

In [7]:
index.fit(documents)

In [8]:
boost = {'question': 3.0 , 'section' : 0.5}

results = index.search(
    query = q,
    boost_dict = boost,
    num_results = 5
)

results 

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the cour

In [9]:
boost = {'question': 3.0 , 'section' : 0.5}

results = index.search(
    query = q,
    boost_dict = boost,
    filter_dict = {'course' : 'data-engineering-zoomcamp'},
    num_results = 5
)

In [10]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

In [11]:
#  from openai import OpenAI
# client = OpenAI()

In [12]:
import os 
from groq import Groq 
from dotenv import load_dotenv 
load_dotenv()

client = Groq(api_key = os.environ.get('GROQ_API_KEY'))



In [13]:
response = client.chat.completions.create(
    model= 'llama3-70b-8192' ,
    messages= [{"role": "user", "content":q}]

)

response.choices[0].message.content


"I'm happy to help!\n\nThe answer to your question depends on the course and the institution offering it. Here are some possible scenarios:**\n\n**Scenario 1: Online course with flexible enrollment**\nIf the course is online and has a flexible enrollment policy, you might still be able to enroll, even if the course has already started. In this case, you can contact the course instructor or the institution's support team to ask about late enrollment. They might allow you to join the course, and you can catch up on any missed material.\n\n**Scenario 2: Traditional on-campus course**\nIf the course is taught on campus and has a fixed schedule, it's less likely that you can still enroll after the course has started. In this case, you may need to wait until the next semester or academic term to enroll.\n\n**Scenario 3: Course with a waitlist**\nIf the course is full, but there's a waitlist, you can still add your name to the waitlist. If a spot becomes available, you'll be notified, and you

In [14]:
prompt_template = """ 
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer, output NONE

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [15]:
context = ""

for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    

In [16]:
prompt = prompt_template.format(question=q, context=context).strip()

In [17]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer, output NONE

QUESTION: the course has already started, can I still enroll?

CONTEXT:
section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your fin

In [18]:
response = client.chat.completions.create(
    model= 'llama3-70b-8192',
    messages = [{"role": "user", "content": prompt}]
)

response.choices[0].message.content 

'Yes'

### Modular Code or Clean Code (Functional Programming)

In [19]:
def search(query):
    boost = {'question': 3.0 , 'section' : 0.5}

    results = index.search(
        query = query,
        boost_dict = boost,
        num_results = 5
    )

    return results 

In [20]:
def build_prompt(query, search_results):
    prompt_template = """ 
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer, output NONE

QUESTION: {question}

CONTEXT:
{context}
""".strip()

    context = ""

    for doc in results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    
    return prompt 

In [21]:
def llm(prompt):
    response = client.chat.completions.create(
        model= 'llama3-70b-8192',
        messages= [{"role": "user", "content": prompt}]
    )

    return response.choices[0].message.content

In [22]:
query = 'the course already started. Can I still enroll?'


def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [23]:
rag(query)

'The answer is YES.'

In [24]:
### USE ElasticSearch Build RAG

In [25]:
from elasticsearch import Elasticsearch

In [26]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [27]:
es_client = Elasticsearch(
    ['http://localhost:9200'],
    headers={
        "Accept": "application/vnd.elasticsearch+json; compatible-with=8",
        "Content-Type": "application/json"
    }  
)

In [28]:
print(es_client)

<Elasticsearch(['http://localhost:9200'])>


In [29]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [30]:
from tqdm.auto import tqdm 

In [31]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [33]:
query = 'the course already started. Can I still enroll?'

In [34]:
def elastic_search(query):

    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index= index_name, body= search_query)

    result_doc = []

    for hit in response['hits']['hits']:
        result_doc.append(hit['_source'])

    return result_doc

In [35]:

def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

rag(query)

In [36]:
rag(query)

'Yes'